# 3.7.1. Clustering Espacial Basado en Densidad de Aplicaciones con Ruido

## Preparación del Entorno

### Carga de Módulos

In [1]:
import math
import os
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import offsetbox
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.figure_factory as ff
# import session_info
from time import time
from plotly.subplots import make_subplots
from sklearn import set_config
from sklearn.datasets import make_blobs

# Tema Principal
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons
import sys

# sys.path.append('../scripts')
from funny_stuffs import calculate_kn_distance, plot_kn_distance

### Configuración Inicial

In [2]:
random_seed = 333  # Semilla para reproducibilidad de resultados
np.random.seed(random_seed)  # Para reproducibilidad

# Configuración de opciones de visualización para pandas
pd.set_option('display.max_columns', None)  # Muestra todas las columnas
pd.set_option('display.max_rows', 15)  # Ajusta el número de filas a mostrar

In [ ]:


# Configuraciones extras
sns.set_style('dark')
dark_template = pio.templates['plotly_dark'].to_plotly_json()
dark_template['layout']['paper_bgcolor'] = 'rgba(30, 30, 30, 0.5)'
dark_template['layout']['plot_bgcolor'] = 'rgba(30, 30, 30, 0.5)'
pio.templates['plotly_dark_semi_transparent'] = go.layout.Template(dark_template)
pio.templates.default = 'plotly_dark_semi_transparent'
set_config(transform_output="pandas")
set_config(display='diagram')
warnings.filterwarnings("ignore")
%matplotlib inline

## Clustering Espacial Basado en Densidad de Aplicaciones con Ruido

### Fundamento Teórico

#### 1. **Definiciones**

DBSCAN  (*Density-Based Spatial Clustering of Applications with Noise*), es un algoritmo de agrupamiento basado en densidad de puntos, especialmente cuando estos grupos están separados por regiones de baja densidad.

Dado un conjunto de entrenamiento $S_n = \{x_i; i = 1, 2, \ldots, n\}$, y una métrica de distancia $d$, necesitamos definir varios conceptos utilizados por el algoritmo DBSCAN, a saber:


**Definición 1 (Eps-Vecindad - MinPts).** La Eps-vecindad de un punto $x$, denotada $N_{\text{Eps}}(x)$ está dada por:
$$ N_{\text{Eps}} (x) = \{ y \in S_n | d(x, y) \leq \text{Eps} \} $$

Dada la existencia de dos tipos de puntos en un cluster, a saber: puntos dentro del cluster (puntos internos) y puntos en la frontera del cluster (puntos umbral), será necesario que para cada punto $x$ en el cluster $C$ exista un punto $y$ en $C$ tal que $x$ se encuentra en la Eps-vecindad de $y$ y $N_{\text{Eps}}(y)$ contenga al menos $\text{MinPts}$, lo anterior se presenta en la siguientes definiciones:

**Definición 2 (Directamente alcanzable en densidad).** Un punto $x$ es directamente alcanzable en densidad desde un punto $y$ con respecto a $\text{Eps}$, $\text{MinPts}$ si:

1. $x \in N_{\text{Eps}} (y)$
2. $|N_{\text{Eps}} (y)| \geq \text{MinPts}$ (condición de punto interno/central).

**Definición 3 (Alcanzable en densidad).** Un punto $x$ es alcanzable en densidad desde un punto $y$ con respecto a $\text{Eps}$, $\text{MinPts}$ si existe una secuencia de puntos $x^{(1)}, \ldots, x^{(n)}, x^{(1)} = y$, $x^{(n)} = x$ tal que cada punto $x^{(1+i)}$ es directamente alcanzable en densidad desde $x^{(i)}$.

**Definición 4 (Conectado en densidad).** Un punto $x$ se dice conectado en densidad a un punto $y$ con respecto a $\text{Eps}$ y $\text{MinPts}$ si existe un punto $z$ tal que tanto $x$ como $y$ son alcanzables en densidad desde $z$ con respecto a $\text{Eps}$ y $\text{MinPts}$.

**Definición 5 (Cluster).** Sea $S_n = \{ x_i \ | \ i = 1, 2, \ldots, n \}$ un conjunto, un cluster $C$ con respecto a $\text{Eps}$ y $\text{MinPts}$ es un conjunto no vacío de $S_n$ que satisface lo siguiente:
1. $\forall x, y$ si $x \in C$ y $y$ es alcanzable en densidad desde $x$ con respecto a $\text{Eps}$ y $\text{MinPts}$, entonces $y \in C$ (Maximalidad).
2. $\forall x, y \in C$: $x$ es conectado en densidad a $y$ con respecto a $\text{Eps}$ y $\text{MinPts}$ (Conectividad).

**Definición 6 (Ruido).** Sean $C_1, \ldots, C_k$ los clusters del conjunto $S_n$ con respecto a los parámetros $\text{Eps}$, $\text{MinPts}$, $i = 1, \ldots, k$, se define el ruido como el conjunto de puntos en $S_n$ que no pertenecen a ningún cluster $C_i$; es decir: ruido = $\{ x \in S_n \ | \ x \not\in C_i \}$.

Con lo anterior podemos establcer el algoritmo DBSCAN:

#### 2. **Algoritmo**

1. **Preparación**
  - **Entrada**: Conjunto de datos $S_n$, parámetros $\text{Eps}$ (epsilon) y $\text{MinPts}$.
  - **Inicialización**: Todos los puntos en $S_n$ están inicialmente marcados como no visitados.

2. **Proceso Principal**
Para cada punto $x$ en el conjunto de datos $S_n$, se repiten los siguientes subpasos hasta que todos los puntos hayan sido visitados:

  - 2.1. **Verificación de Punto No Visitado**
    - Si $x$ ya ha sido visitado, se pasa al siguiente punto en el conjunto de datos. Si $x$ no ha sido visitado, se marca como visitado y se procede al siguiente subpaso.

  - 2.2.  **Búsqueda de Vecinos**
    - Se calcula la ε-vecindad de $x$, $N_{\epsilon}(p)$, que incluye todos los puntos dentro de un radio $\text{Eps}$ de $x$.

  -  2.3. **Evaluación de Punto Central**
    - Si $|N_{\epsilon}(p)|$ es menor que $\text{MinPts}$, $x$ se marca como ruido (temporalmente, ya que podría ser alcanzado por un punto central en un paso posterior). Si $|N_{\epsilon}(x)|$ es mayor o igual a $\text{MinPts}$, $x$ es un punto central, y se procede al siguiente subpaso.

  - 2.4. **Expansión de Grupos**
    - Para un punto central $x$, se crea un nuevo grupo incluyendo $x$ y todos los puntos en $N_{\epsilon}(p)$.
    - Para cada punto $y$ en $N_{\epsilon}(p)$:
      - Si $y$ es un punto no visitado, se marca como visitado y se verifica si es un punto central (es decir, si $|N_{\epsilon}(q)|$ es mayor o igual a $\text{MinPts}$). Si es así, la ε-vecindad de $y$ se añade al grupo.
      - Si $y$ no pertenece a ningún grupo, se añade al grupo actual.

3. **Asignación de Puntos Frontera y Ruido**
  - A lo largo del proceso de expansión de grupos, los puntos que no son centrales pero se encuentran en la vecindad de un punto central se identifican como puntos frontera.
  - Los puntos que no han sido incluidos en ningún grupo después de explorar todo el conjunto de datos se consideran ruido definitivo.

4. **Resultado**
  Al finalizar, el algoritmo habrá identificado varios grupos basados en la densidad de los puntos en el conjunto de datos. Cada punto pertenecerá a exactamente un grupo o será marcado como ruido.

In [3]:
# DBSCAN: Diferentes Parámetros
X, _ = make_moons(n_samples=300, noise=0.05, random_state=42)

dbscan1 = DBSCAN(eps=0.1, min_samples=5)
clusters1 = dbscan1.fit_predict(X)
dbscan2 = DBSCAN(eps=0.2, min_samples=10)
clusters2 = dbscan2.fit_predict(X)

fig = go.Figure()

fig.add_trace(go.Scatter(x=X[:, 0], y=X[:, 1], mode='markers',
                        marker=dict(size=8, color=clusters1, colorscale='Viridis', line_width=1),
                        name='EPS=0.1, MinPts=5'))
fig.add_trace(go.Scatter(x=X[:, 0] + 2.5, y=X[:, 1], mode='markers',  # Desplazamos en X para diferenciar
                        marker=dict(size=8, color=clusters2, colorscale='Cividis', line_width=1),
                        name='EPS=0.2, MinPts=10'))
fig.update_layout(title="DBSCAN Clustering con Diferentes Parámetros",
                  xaxis_title="X",
                  yaxis_title="Y",
                  legend_title="Clusters")

# Mostrar figura
fig.show()


#### Aplicaciónes y Consideraciones

<p style="font-size:25px;">Aplicaciones</p>

1. Detección de Anomalías
2. Sistemas de Información Geográfica
3. Segmentación de Imágenes
4. Análisis de Tráfico
5. Biología y Ecología


<p style="font-size:25px;">Ventajas</p>

1. No requiere Especificación de Número de Grupos
2. Maneja Grupos de Diversas Formas y Tamaños
3. Robusto ante Ruido y Puntos Atípicos
4. Requiere Pocos Parámetros


<p style="font-size:25px;">Consideraciones</p>

1. Determinación de Parámetros
2. Sensibilidad a Parámetros
3. No adecuado para Datos de Alta Dimensión
4. Variedad en la Densidad de Grupos
5. Complejidad Computacional
6. Dependencia de la Métrica de Distancia
7. Necesidad de Preprocesamiento
8. Evaluación de Resultados

### Ejemplo Práctico

#### Preparación de los Datos

**Objetivo**

Segmentar a los clientes de un centro comercial en diferentes grupos basándonos en características como el ingreso anual y la puntuación de gasto. Al comprender las distintas categorías de clientes, la administración del centro comercial podrá diseñar estrategias de marketing más dirigidas para la implentación de una tarjeta departamental.

**Planteamiento del problema**

El centro comercial ha recopilado datos de sus clientes que incluyen el ingreso anual y la puntuación de gasto, pero no tiene una comprensión clara de cómo se distribuyen estos clientes en términos de hábitos de compra y poder adquisitivo. El equipo de marketing necesita una forma de categorizar a los clientes de manera eficiente para poder enfocar sus esfuerzos y recursos en campaña destinada en otorgar una tarjeta departamental a los grupos de mayor valía.

Las características extraídas de cada cliente son:

1. **Ingreso Anual (Annual Income)**: El ingreso anual del cliente en miles de dólares. Esta métrica es un indicador clave del poder adquisitivo de un cliente y puede correlacionarse con su potencial de gasto.

2. **Puntuación de Gasto (Spending Score)**: Una puntuación asignada por el centro comercial basada en el comportamiento de compra del cliente y la frecuencia de sus visitas. Los valores altos indican clientes que compran con más frecuencia y/o gastan más dinero.

Con estos datos, se aplicará un método de agrupamiento no supervisado, para identificar estructuras naturales en el conjunto de datos y clasificar a los clientes en segmentos distintos. Estos segmentos permitirán al equipo de marketing del centro comercial diseñar ofertas personalizadas, optimizar la disposición de las tiendas y mejorar la asignación de recursos para campañas publicitarias. Además, el conocimiento detallado de los segmentos de clientes ayudará a predecir tendencias de mercado y a planificar estratégicamente las expansiones o renovaciones del centro comercial.

In [4]:
# Carga del conjunto de datos
df = pd.read_csv('../data/Mall_Customers.csv')

In [5]:
df.describe()

,CustomerID,Age,Annual Income (k$),Spending Score (1-100)
count,200.000000,200.000000,200.000000,200.000000
mean,100.500000,38.850000,60.560000,50.200000
std,57.879185,13.969007,26.264721,25.823522
min,1.000000,18.000000,15.000000,1.000000
25%,50.750000,28.750000,41.500000,34.750000
50%,100.500000,36.000000,61.500000,50.000000
75%,150.250000,49.000000,78.000000,73.000000
max,200.000000,70.000000,137.000000,99.000000


Ya que no existe una influencia de escala entre las dos características no utilizaremos un método de escalamiento.

In [6]:
fig = px.scatter(df, x='Annual Income (k$)', y='Spending Score (1-100)', 
               title='Ingreso Anual vs. Puntuación de Gasto',
               labels={'Annual Income (k$)': 'Ingreso Anual (k$)', 'Spending Score (1-100)': 'Puntuación de Gasto (1-100)'},
               width=800, height=500)

fig.show()

In [7]:
# Categorizar las edades en grupos
df['Age Group'] = pd.cut(df['Age'], bins=[18, 30, 40, 50, 80], labels=['18-30', '31-40', '41-50', '50-80'], right=False)

# Scatterplot por grupo de edad
fig = px.scatter(df, x='Annual Income (k$)', y='Spending Score (1-100)',
                 color='Age Group',
                 title='Ingreso Anual vs. Puntuación de Gasto por Grupo de Edad',
                 labels={'Annual Income (k$)': 'Ingreso Anual (k$)', 'Spending Score (1-100)': 'Puntuación de Gasto (1-100)'},
                 color_discrete_sequence=px.colors.qualitative.Set1,
                 width=800, height=500)

fig.show()


In [8]:
# Scatterplot por grupo de edad
fig = px.scatter(df, x='Annual Income (k$)', y='Spending Score (1-100)',
                  color='Genre',
                  title='Ingreso Anual vs. Puntuación de Gasto por Grupo de Genero',
                  labels={'Annual Income (k$)': 'Ingreso Anual (k$)', 'Spending Score (1-100)': 'Puntuación de Gasto (1-100)'},
                  color_discrete_sequence=px.colors.qualitative.Set1,
                  width=800, height=500)

fig.show()

Al no tener relaciones aparentes nos centraremos solo en las variables que son más importantes: 'Annual Income (k$)' y 'Spending Score (1-100)'

In [9]:
# Preparamos las características relevantes
X = df[['Annual Income (k$)', 'Spending Score (1-100)']]
#X_scaled = StandardScaler().fit_transform(X)

plot_kn_distance(X, 5)

Con esto podemos notar que para un k cercano a 5 el cambio se da en (167,10.29)

#### Implementación del Método

Principales parámetros de $DBSCAN$:
```python
DBSCAN(
    eps=0.5,
    min_samples=5,
    metric='euclidean',
)

#### Visualización e Implementación de Resultados

In [10]:
# Aplicamos DBSCAN
dbs = DBSCAN(eps=10.5, min_samples=5).fit(X)
labels = dbs.labels_

df['Cluster'] = labels

fig = px.scatter(df, x='Annual Income (k$)', y='Spending Score (1-100)', color=labels.astype(str),
               title='DBSCAN Clustering de Clientes del Centro Comercial',
               labels={'color': 'Cluster'},
               color_discrete_sequence=px.colors.qualitative.Set1)
fig.show()

Realizaremos el perfilamiento de los grupos, omitiendo el ruido (etiquetas igual a -1):

In [11]:
# Filtramos los datos para omitir el ruido
clusters_df = df[df['Cluster'] != -1]

# Realizamos perfilamiento para cada grupo
profile = clusters_df.groupby('Cluster').agg(
    Count=('CustomerID', 'size'),
    Average_Age=('Age', 'mean'),
    Average_Annual_Income=('Annual Income (k$)', 'mean'),
    Average_Spending_Score=('Spending Score (1-100)', 'mean')
).reset_index()

profile.round()

,Cluster,Count,Average_Age,Average_Annual_Income,Average_Spending_Score
0,0,115,40.0,48.0,52.0
1,1,11,49.0,24.0,9.0
2,2,32,33.0,81.0,84.0
3,3,27,41.0,84.0,14.0


**Perfilamiento:**

1. **Grupo 0**: Este grupo es el más numeroso, con 115 clientes. Tienen una edad promedio de aproximadamente 40 años, un ingreso anual promedio de alrededor de $48k, y un puntaje de gasto promedio cercano a 52. Este grupo representa a clientes de mediana edad con ingresos y gastos moderados.
2. **Grupo 1**: Este es el grupo más pequeño, con solo 11 clientes. Presentan una edad promedio de casi 50 años, el ingreso anual promedio más bajo de alrededor de $24k, y un puntaje de gasto promedio muy bajo de aproximadamente 9. Estos son clientes mayores con ingresos bajos y muy poco gasto.
3. **Grupo 2**: Con 32 clientes, este grupo tiene una edad promedio de unos 33 años, un ingreso anual promedio alto de aproximadamente $81k, y el puntaje de gasto promedio más alto de alrededor de 84. Estos clientes son relativamente jóvenes, con altos ingresos y un alto nivel de gasto.
4. **Grupo 3**: Este grupo incluye 27 clientes con una edad promedio de alrededor de 41 años, el ingreso anual promedio más alto de cerca de $84k, y un puntaje de gasto promedio bajo de aproximadamente 14. Son clientes de mediana edad con altos ingresos pero bajos niveles de gasto.


**Estrategia para Ofrecer Tarjetas Departamentales**

Basado en el perfilamiento, se recomendaría enfocar la oferta de tarjetas departamentales en los siguientes grupos:

- **Grupo 2**: Dado su alto ingreso y alto nivel de gasto, este grupo es ideal para tarjetas departamentales. Estos clientes tienen el potencial de aprovechar los beneficios y ofertas que suelen acompañar a este tipo de tarjetas, aumentando aún más su gasto.
- **Grupo 0**: Aunque su nivel de gasto no es tan alto como en el Grupo 2, su ingreso moderado y la cantidad de clientes en este grupo lo hacen un buen candidato para ofertas de tarjetas departamentales, posiblemente con promociones especiales para aumentar su gasto.

**Grupo a al que No Ofrecería Tarjetas Departamentales:**
- **Grupo 1**: Con ingresos y gastos bajos, este grupo podría no beneficiarse tanto de las tarjetas departamentales, y podrían ser menos propensos a usarlas activamente.

**Grupo a incentivar:**
- **Grupo 3**: A pesar de sus altos ingresos, su bajo nivel de gasto sugiere que podrían no estar interesados en las compras que las tarjetas departamentales suelen incentivar.

**Conclusión**

Concentrar los esfuerzos en los grupos 2 y 0 maximizará las oportunidades de ventas incrementales y la lealtad del cliente a través de beneficios y recompensas asociadas con el uso de la tarjeta departamental.

Este ejercicio demostró cómo aplicar con éxito el modelo DBSCAN, si bien no es tan intuitivo definir los hiperparámetros, los resultados de una buena aplicación son evidentes, más aún que no tenemos que definir un numero de grupos (no sesgando nuestra decisión del número de clusters) y manteniendo los valores atípicos fuera del análisis.

Es importante recordar que como estamos utilizando un modelo basado en distancias, se debe tomar en cuenta si debemos escalar o no nuestro conjunto de datos.